In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [13]:
# gdf_locations = gpd.read_file('../joined-data/simplified_matches_4326.geo.json')

df_matches_all = pd.read_csv('../joined-data/simplified_matches_uw_tenure_full_cra.csv')
gdf_locations = gpd.GeoDataFrame(
    df_matches_all, geometry=gpd.points_from_xy(df_matches_all.X_Coordinate, df_matches_all.Y_Coordinate), crs="EPSG:4326"
)
gdf_locations = gdf_locations[[
    'CRA_BN_ID', 'CRA_Organization_Name', 'CRA_Category', 'CRA_SubCategory',
    '211_Organization_Name', '211_Location_Name', '211_Address', '211_City',
    '211_Postal_Code', 'X_Coordinate', 'Y_Coordinate', 'Tenure', 'UW']]

def assign_region(city):
    if city == 'Toronto':
        return 'Toronto'
    elif city in ['Brampton', 'Mississauga', 'Caledon']:
        return 'Peel'
    else:
        return 'York'

gdf_locations['region'] = gdf_locations['211_City'].apply(assign_region)

gdf_ct = gpd.read_file('../equity-data/ct-data-all.geo.json')

Finding 3: Average number of agency spaces (ie. individual locations) within a census tract area (ie. completely within a census tract or no more than 800 meters away from it). We want to break this down by each of the quantiles of the equity index as well.

Finding 4: Percentage of agency spaces near existing and future transit stations (ie. within 800m). We want this broken down by owned (CORE) and leased (CLRE)

Finding 5: Percentage of agency spaces which are rented (CLRE), broken down by municipality and equity index quintile